Passo 1: Instalar Dependências
Antes de tudo, precisamos instalar os pacotes necessários para o ambiente do modelo e do sintetizador.

In [ ]:
%pip install torch==1.0.1.post2 torchvision==0.2.2.post3
%pip install pretty_midi

Passo 2: Clonar o Repositório do LakhNES
Agora, baixe o código-fonte do LakhNES:

In [ ]:
%git clone https://github.com/chrisdonahue/LakhNES.git
%cd LakhNES

Passo 3: Configurar o Ambiente do Modelo
O LakhNES requer um ambiente separado para o modelo. Configure um ambiente virtual dentro do Jupyter:

In [ ]:
%python -m venv LakhNES-model
%source LakhNES-model/bin/activate

Instale os pacotes dentro desse ambiente:

In [ ]:
%pip install torch==1.0.1.post2 torchvision==0.2.2.post3


Se estiver no Windows, ative o ambiente com:

In [ ]:
%LakhNES-model\Scripts\activate

Passo 4: Configurar o Ambiente do Sintetizador
O sintetizador precisa do Python 2.7 (não compatível com Python 3). Configure um ambiente virtual separado:

In [ ]:
%virtualenv -p python2.7 LakhNES-synth
%source LakhNES-synth/bin/activate
%pip install nesmdb pretty_midi

Se estiver no Windows, ative o ambiente com:

In [ ]:
%LakhNES-synth\Scripts\activate


Em seguida, inicie o servidor de síntese de áudio:

In [ ]:
%python data/synth_server.py 1337


Passo 5: Baixar os Modelos Pré-Treinados
Agora, baixe os checkpoints pré-treinados para geração de músicas. O recomendado é o LakhNES 400k:

In [ ]:
%wget -O model/LakhNES.pth "https://drive.google.com/uc?export=download&id=1ND27trP3pTAl6eAk5QiYE9JjLOivqGsd"

Passo 6: Gerar uma Música Chiptune
Agora, gere uma nova música chiptune usando o modelo:

In [ ]:
%python generate.py model/LakhNES.pth --out_dir ./generated --num 1


Converta para áudio usando o sintetizador:

In [ ]:
%python data/synth_client.py ./generated/0.tx1.txt ./generated/0.tx1.wav


Reproduza o áudio gerado:

In [ ]:
%aplay ./generated/0.tx1.wav

Passo 7: Teste com uma Música Conhecida
Se quiser testar com uma música do Kirby's Adventure, faça:

In [ ]:
%python data/synth_client.py data/nesmdb_tx1/train/191_Kirby_sAdventure_02_03PlainsLevel.tx1.txt kirby_tx1.wav 48
%aplay kirby_tx1.wav

### AGORA QUE TEMOS O SINTETIZADOR FUNCIONANDO, VAMOS FAZER O PROCESSAMENTO PARA CASOS GERAIS DE AUDIOS EM .wav e .mp3

1. Carregar e Pré-processar o Áudio
Converter o arquivo para um formato adequado (wav com taxa de amostragem fixa).
Normalizar o áudio e converter para mono.
Opcionalmente, aplicar filtragem para remoção de ruídos.

In [ ]:
from pydub import AudioSegment
import librosa

def load_audio(file_path):
    audio = AudioSegment.from_file(file_path).set_channels(1).set_frame_rate(44100)
    samples = audio.get_array_of_samples()
    return samples, audio.frame_rate
    

4. Extrair Frequências Dominantes
Utilizar a Transformada de Fourier (FFT) para obter a distribuição espectral do áudio.
Identificar os componentes mais relevantes, separando melodia, baixo e percussão.

In [ ]:
import numpy as np
import librosa.display
import matplotlib.pyplot as plt

def compute_fft(samples, sr):
    stft = np.abs(librosa.stft(samples, n_fft=2048, hop_length=512))
    freqs = librosa.fft_frequencies(sr=sr)
    
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(librosa.amplitude_to_db(stft, ref=np.max),
                             y_axis='log', x_axis='time')
    plt.colorbar()
    plt.title("Espectrograma")
    plt.show()
    
    return freqs, stft

5. Detectar Notas Musicais
Utilizar algoritmos de pitch detection (Autocorrelação, HPS) para extrair notas.
Converter as frequências em notas MIDI.

In [ ]:
def pitch_detection(samples, sr):
    pitches, magnitudes = librosa.piptrack(y=samples, sr=sr)
    pitch_values = np.max(pitches, axis=0)
    return pitch_values

6. Reduzir para 4 Canais
Aplicar clustering (K-means) para agrupar as frequências mais relevantes em 4 categorias:
Canal 1 e 2: Ondas pulsadas (melodia/harmonia)
Canal 3: Onda triangular (baixo)
Canal 4: Ruído (percussão)

In [ ]:
from sklearn.cluster import KMeans

def reduce_polyphony(frequencies):
    kmeans = KMeans(n_clusters=4, random_state=0).fit(frequencies.reshape(-1, 1))
    return kmeans.labels_

7. Criar Arquivo MIDI
Gerar um arquivo MIDI compatível com os 4 canais do NES.

In [ ]:
from mido import Message, MidiFile, MidiTrack

def create_midi(notes, output_file):
    midi = MidiFile()
    track = MidiTrack()
    midi.tracks.append(track)

    for note in notes:
        track.append(Message('note_on', note=note, velocity=64, time=0))
        track.append(Message('note_off', note=note, velocity=64, time=480))

    midi.save(output_file)

8. Ajustar Timing (Quantização Temporal)
Ajustar a duração das notas conforme o clock do NES.
Reduzir notas rápidas para evitar sobrecarga.

9. Testar e Ajustar a Ferramenta
Comparar a saída MIDI com a música original.
Aplicar filtros para melhorar a transcrição.
Verificar a fidelidade da conversão e ajustar o algoritmo.

10. Criar Interface (Opcional)
Criar uma interface simples em Python para o usuário carregar arquivos e baixar a conversão.